In [14]:
import pandas as pd
from pymer4.models import Lmer

from load_feat_pd import *
import matplotlib.pyplot as plt
import seaborn as sns


def lme(metadata, outfile, featname='shimmer', level='utt', stats='mean'):

    df = pd.DataFrame(metadata)
    
    # remove group id == '11':
    df = df[df['group_id'] != '11']
    
    # map group_id 21 to HC 22 to PD
    df['group_id'] = df['group_id'].map({'21': 'HC', '22': 'PD'})


    # change age from string to int
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    missing_age = df['age'].isnull().sum()
    if missing_age > 0:
        print(f"warning 'age' has {missing_age} NaN, has been deleted")
        df = df.dropna(subset=['age'])
        
    # add one more column item name in the filename
    df['item'] = df['filename'].apply(lambda x: x.split('.')[0].split('_')[-1])
    
    # df['group_id'] = pd.Categorical(df['group_id'], categories=['PD', 'Control'])
    df['gender'] = pd.Categorical(df['gender'], categories=['V', 'M'])
    # df['experiment'] = pd.Categorical(df['experiment'], categories=['exp_1_PictureNaming', 'exp_2_EarlyLate', 'exp_3_BoundaryTone'])

    # print("Group ID Levels:", df['group_id'].cat.categories)       # 输出: ['PD', 'Control']
    # print("Gender Levels:", df['gender'].cat.categories)           # 输出: ['Female', 'Male']
    # print("Experiment Levels:", df['experiment'].cat.categories)   # 输出: ['Baseline', 'EarlyLate', 'BoundaryTone']

    print(df.head())


    if level == 'utt':
        # change the value from np array to float 
     
        if type(df['value'].iloc[0]) == np.ndarray:
            df['value'] = df['value'].apply(lambda x: float(x))
        
    elif level == 'frame':
        if stats == 'mean':
            df['value'] = df['value'].apply(lambda x: x.mean())
        elif stats == 'std':
            df['value'] = df['value'].apply(lambda x: x.std())
        elif stats == 'median':
            df['value'] = df['value'].apply(lambda x: np.median(x))

    # print(df.head())
    # print(df.shape)
    featname = "response_time" if featname == 'rp' else featname
    valuetype = stats if level == 'frame' else ''

    print(f'=============== value: {featname}_{valuetype} ====================\n')


    model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
                        

    result = model.fit()

    
        
    with open(outfile, 'a') as f:
        f.write(f'\n')
        f.write(f'=============== value: {featname}_{valuetype} ====================\n')
        f.write(result[['Estimate', 'SE', 'T-stat', 'P-val', 'Sig']].to_string())    
        f.write(f'\n')
        f.write(model.ranef_var.to_string())
        f.write(f'\n')
    
    print(result[['Estimate', 'SE', 'T-stat', 'P-val', 'Sig']])

    return df


    """
    Visualizes the specified feature with respect to group (PD vs HC) and gender (Female vs Male).
    
    Parameters:
    - df: pandas DataFrame containing the data.
    - featname: str, name of the feature to visualize.
    - outfile: str, path to save the visualization.
    - significance: bool, whether to add significance annotations.
    """
    # Map gender codes to labels
    df['gender'] = df['gender'].map({'V': 'Female', 'M': 'Male'})
    
    # Set the aesthetic style of the plots
    sns.set(style="whitegrid")
    
    plt.figure(figsize=(8, 6))
    
    # Bar Plot with error bars (standard deviation)
    ax = sns.barplot(x='gender', y='value', hue='group_id', data=df, ci='sd', palette='muted')
    
    plt.title(f'{featname.capitalize()} by Group and Gender')
    plt.xlabel('Gender')
    plt.ylabel(featname.capitalize())
    
    # Add individual data points
    sns.stripplot(x='gender', y='value', hue='group_id', data=df, 
                  dodge=True, color='black', alpha=0.5, size=3)
    
    # Adjust the legend to avoid duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        plt.legend(handles[:2], labels[:2], title='Group')
    
    
    plt.tight_layout()
    plt.savefig(outfile, dpi=300)
    plt.show()
    
    print(f"{featname.capitalize()} visualization saved to '{outfile}'")


def visualize_feat_boxplot(df, featname, outfile):
    """
    Visualizes the specified feature with respect to group (PD vs HC) and gender (Female vs Male) using a box plot.
    
    Parameters:
    - df: pandas DataFrame containing the data.
    - featname: str, name of the feature to visualize (used for labeling).
    - outfile: str, path to save the visualization.
    """
    # Map gender codes to labels
    df['gender'] = df['gender'].map({'V': 'Female', 'M': 'Male'})
    
    # Verify that 'value' column exists
    if 'value' not in df.columns:
        raise ValueError("The DataFrame does not contain a 'value' column.")
    
    # Set the aesthetic style of the plots
    sns.set(style="whitegrid")
    
    plt.figure(figsize=(8, 6))
    
    # Box Plot using 'value' as the y-axis
    ax = sns.boxplot(
        x='gender',
        y='value',            # Use 'value' instead of featname
        hue='group_id',
        data=df,
        palette='muted',
        showfliers=False     # Optional: hide outliers
    )
    
    plt.title(f'{featname.capitalize()} by Group and Gender')
    plt.xlabel('Gender')
    plt.ylabel(featname.capitalize())
    
    # Add individual data points
    sns.stripplot(
        x='gender',
        y='value',            # Use 'value' instead of featname
        hue='group_id',
        data=df,
        dodge=True,
        color='black',
        alpha=0.5,
        size=3
    )
    
    # Adjust the legend to avoid duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        plt.legend(handles[:2], labels[:2], title='Group')
    
    plt.tight_layout()
    plt.savefig(outfile, dpi=300)
    plt.show()
    
    print(f"{featname.capitalize()} visualization saved to '{outfile}'")




        
if __name__ == '__main__':
    base_folder_path_unnorm = Path('/data/storage500/Turntaking/wavs_single_channel_nosil')
    base_folder_path = Path('/data/storage500/Turntaking/wavs_single_channel_normalized_nosil')

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame',
        'spec_cent': 'frame',
        'spec_bw': 'frame',
        'spec_rolloff': 'frame',
        'spec_rolloff_min': 'frame',
        'zcr': 'frame',
        'flatness': 'frame',
        'F1': 'frame',
        'F2': 'frame',
        'F3': 'frame',
        'hnr_value': 'utt',
        'contrast': '3d',

    }

    np.set_printoptions(precision=2)
    allfeats = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    allfeats = ['F1', 'F2', 'F3', 'zcr', 'flatness', 'hnr_value','spec_cent', 'spec_bw', 'spec_rolloff', 'spec_rolloff_min', 'contrast']
    
    outfile = './res_lme/res_v1_V_batch2.txt'
    # if exist, remove the file
    if os.path.exists(outfile):
        os.remove(outfile)
    
    feature_dfs = {}  # Dictionary to store DataFrames for each feature
    
    for featname in allfeats:
        folder = base_folder_path_unnorm if featname == 'energy' else base_folder_path

        if feats2level[featname] == 'frame':
            metadata = load_feat(folder, feature_name=featname)
            df = lme(metadata, outfile, featname=featname, level=feats2level[featname], stats='mean')
            feature_dfs[featname + '_mean'] = df

            df = lme(metadata, outfile, featname=featname, level=feats2level[featname], stats='std')
            # Optionally, process 'std' or other statistics
            feature_dfs[featname + '_std'] = df
            
        elif feats2level[featname] == 'utt':
            metadata = load_feat(folder, feature_name=featname)
            df = lme(metadata, outfile, featname=featname, level=feats2level[featname])
            feature_dfs[featname] = df
        
        elif feats2level[featname] == '3d':
            metadatas = load_feat(folder, feature_name=featname, threeD=True)
            for i, metadata in enumerate(metadatas):
                df = lme(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='mean')
                feature_dfs[f'{featname}_{i}_mean'] = df
                df = lme(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='std')
                feature_dfs[f'{featname}_{i}_std'] = df
    
        
    # for featname in feature_dfs.keys():
    #     visualize_feat_boxplot(feature_dfs[featname], featname, f'./res_lme/{featname}_visualization.png')



Processing PictureNaming folder...
Found 1652 npy files
0 files with all 0 values
Processing EarlyLate folder...
Found 3971 npy files
nan in /data/storage500/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/F1/subj-2212_45_L_toetsen_draagbaar.wav_1.wav_F1.npy
0 files with all 0 values
Processing BoundaryTone folder...
Found 5026 npy files
0 files with all 0 values
warning 'age' has 156 NaN, has been deleted
            experiment group_id  \
1  exp_1_PictureNaming       HC   
2  exp_1_PictureNaming       PD   
3  exp_1_PictureNaming       PD   
4  exp_1_PictureNaming       PD   
5  exp_1_PictureNaming       PD   

                                               value subject_id  \
1  [358.1099986350024, 396.2524513859543, 451.278...       2116   
2  [2005.5827789983457, 291.84701344002224, 961.5...       2208   
3  [1024.3481408169682, 1141.7095343981666, 244.7...       2205   
4  [666.5808400105267, 797.6304891259256, 649.673...       2203   
5  [2407.0145952758717, 2